# Data walking through

In [1]:
import pandas as pd 
import numpy as np

# read csv and assign columns
columns = ["personId", "productId"]
df = pd.read_csv("dataset/ratebeer/wish.csv", header=None)
df.columns = columns

number_of_row = len(df)
# get number of products
number_of_product = len(np.unique(df["productId"]))
# get number of person
number_of_person = len(np.unique(df["personId"]))
df = df.groupby("personId").aggregate(lambda x: list(x))

In [21]:
columns = ["personId", "productId"]
have_df = pd.read_csv("dataset/ratebeer/have.csv", header=None)
have_df.columns = columns
have_df = have_df.groupby("personId").aggregate(lambda x: list(x))
have_df.head()

,productId
personId,
39,[102296]
69,"[1195, 5923, 37101, 31915]"
154,"[2490, 2487, 38037, 24021, 7223, 22427, 32152,..."
189,"[3063, 38037, 3015, 3062, 8591, 3013, 3014, 30..."
605,"[164, 5400, 30896, 371, 30812, 22905]"


In [22]:
len(have_df)

1806

In [2]:
print("Number of row:", number_of_row)
print("Number of person:", number_of_person)
print("Number of product:", number_of_product)
# first samples
df.head()

Number of row: 118453
Number of person: 1953
Number of product: 21654


,productId
personId,
39,"[47436, 1147, 64658, 9448, 12227, 10609, 6890,..."
63,"[36407, 53647, 36624]"
69,"[15017, 2491, 6924, 9193, 19633, 13730, 64975,..."
154,"[49774, 28173, 15017, 24350, 565, 22339, 17385..."
181,"[10524, 30591, 7183, 38426, 22537, 48679, 2912..."


In [105]:
def transactionEncoder(df):
    # 'transactions' is now temporary variable
    transactions = [row["productId"] for index, row in df.iterrows()]
    from mlxtend.preprocessing import TransactionEncoder

    transaction_encoder = TransactionEncoder()
    wish_matrix = transaction_encoder.fit_transform(transactions).astype("int")
    wish_df = pd.DataFrame(wish_matrix, columns=transaction_encoder.columns_)

    return wish_df, wish_matrix

In [3]:
# 'transactions' is now temporary variable
transactions = [row["productId"] for index, row in df.iterrows()]

In [4]:
from mlxtend.preprocessing import TransactionEncoder

transaction_encoder = TransactionEncoder()
wish_matrix = transaction_encoder.fit_transform(transactions).astype("int")
wish_df = pd.DataFrame(wish_matrix, columns=transaction_encoder.columns_)

In [5]:
wish_df.head()

,1,2,3,9,10,13,14,15,16,17,...,390987,391005,391163,391340,391802,392092,392165,392402,392478,393042
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
np.unique(wish_df.sum(), return_counts=True)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  91,  92,
         93,  94,  95,  96,  98,  99, 100, 101, 102, 103, 105, 106, 107,
        109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
        122, 123, 124, 125, 126, 128, 131, 132, 134, 135, 136, 137, 140,
        141, 142, 143, 144, 145, 146, 148, 149, 151, 153, 154, 158, 160,
        161, 164, 165, 167, 169, 170, 173, 174, 175, 178, 180, 183, 184,
        185, 188, 189, 194, 197, 199, 201, 202, 203, 205, 207, 210, 221,
        228, 231, 233, 240, 242, 246, 248, 254, 300

# Transaction-based wish prediction using apriori

In [14]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(wish_df, min_support=0.15)
print(frequent_itemsets)
rules = association_rules(frequent_itemsets, metric="confidence")
# Something FAILED
rules

    support itemsets
0  0.167947   (2877)
1  0.177675   (4032)
2  0.183820   (5202)
3  0.218126   (6441)
4  0.153610   (6474)
5  0.182284   (6965)
6  0.178699   (7080)
7  0.191500   (8021)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [106]:
wish_have_df, wish_have_matrix = transactionEncoder(_df)

frequent_itemsets = apriori(wish_have_df, min_support=0.2)
print(frequent_itemsets)
rules = association_rules(frequent_itemsets, metric="confidence")
# Something FAILED
rules

    support itemsets
0  0.204515   (1429)
1  0.217607   (2388)
2  0.270880   (3566)
3  0.218059   (8223)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


# User-based prediction using cosine similarity

In [119]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(wish_df[1:2], wish_matrix[:])[0][:200]

t = cosine_similarity(wish_df[1:2], wish_matrix[:])[0]
mx = 0
for i in t:
    if i < 1 and i > mx:
        mx = i
mx

0.408248290463863

In [118]:

t = cosine_similarity(wish_have_df[1:2], wish_have_matrix[:])[0]
mx = 0
for i in t:
    if i < 1:
        if i > mx:
            mx = i
mx

0.408248290463863

In [24]:
have_df.head()

,productId
personId,
39,[102296]
69,"[1195, 5923, 37101, 31915]"
154,"[2490, 2487, 38037, 24021, 7223, 22427, 32152,..."
189,"[3063, 38037, 3015, 3062, 8591, 3013, 3014, 30..."
605,"[164, 5400, 30896, 371, 30812, 22905]"


In [25]:
_have_df = have_df
_df = df

In [70]:
df.index.values

array([    39,     63,     69, ..., 367177, 367429, 371199])

In [101]:
_df = df
cnt = 0
for i in range(len(have_df)):
    have_PID = have_df.iloc[i].name
    if have_PID in _df.index.values:
        _df.loc[have_PID][0] = np.append(_df.loc[have_PID][0], have_df.iloc[i][0])
        _df.loc[have_PID][0] = np.unique(_df.loc[have_PID][0])
    else:
        _df = _df.append(have_df.iloc[i])

262